# import

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
from collections import Counter
import tes_analysis_tools as tat

import sklearn
from sklearn.cluster import KMeans, DBSCAN
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

from tsfresh.feature_extraction import extract_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.utilities.distribution import MultiprocessingDistributor

warnings.filterwarnings("ignore")
sns.set()

# utils

In [2]:
# データ補正 <- 18次回帰曲線
def regression(data, time):
    fit = np.polyfit(time, data, 18)
    fit_fn = np.poly1d(fit)
    return fit_fn(time)


# 標準化
def standardize(df):
    scaler = StandardScaler()
    df_std = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
    return df_std


# tsfreshの特徴量に対する前処理
def remove_constant_columns(df):
    df_filtered = df.loc[:, df.nunique() != 1]
    return df_filtered


def remove_binary_columns(df):
    columns_to_select = df.apply(lambda col: not set(col.unique()).issubset({0, 1}))
    df_filtered = df.loc[:, columns_to_select]
    return df_filtered


def remove_constant_or_binary_columns(df):
    columns_to_select = df.apply(lambda col: (col.nunique() != 1) and not set(col.unique()).issubset({0, 1}))
    df_filtered = df.loc[:, columns_to_select]
    return df_filtered


def remove_corr_cols(df, threshold=0.7):
    corr = df.corr().abs()
    high_corr = np.where(corr > threshold)
    to_drop = []
    for idx, col in zip(high_corr[0], high_corr[1]):
        f1, f2 = corr.index[idx], corr.columns[col]
        if f1 != f2 and f1 not in to_drop and f2 not in to_drop:
            to_drop.append(f1)
    df_filtered = df.drop(to_drop, axis=1)
    return df_filtered


def preprocess(df, threshold=0.7):
    df_filtered = remove_constant_or_binary_columns(df)
    df_filtered = standardize(df_filtered)
    df_filtered = remove_corr_cols(df_filtered, threshold)
    return df_filtered


# 主成分分析の特徴量に対する前処理
# 第一主成分の平均値が低いクラスタから0, 1, ...と再ラベリング
def relabel_by_first_component_avg(label_col, pca_df):
    relabeled_pca_df = pca_df.copy()
    first_component_avg_per_cluster = {}
    for label in set(pca_df[label_col]):
        if label != -1:
            first_component_avg_per_cluster[label] = pca_df[pca_df[label_col] == label]["first_component"].mean()
    sorted_keys = sorted(first_component_avg_per_cluster, key=lambda x: first_component_avg_per_cluster[x])
    relabel_dict = {k: v for v, k in enumerate(sorted_keys)}
    relabel_dict[-1] = -1
    relabeled_pca_df[label_col] = relabeled_pca_df[label_col].map(relabel_dict)
    return relabeled_pca_df

# データ読み込み、作成

In [ ]:
# -----------------------
# データ読み込み
# -----------------------

dir = "../WF/exp1/"
savedir = "./exp1_result/"
pulse = np.load(dir + "pulse.npy")
noise = np.load(dir + "noise.npy")
time = np.load(dir + "time.npy")

n = pulse.shape[0]
dp = pulse.shape[1]
m = noise.shape[0]
dt = time[1] - time[0]

# baseline 補正, 先頭からdpbl点の平均をoffsetとして引き去る
dpbl = 200
pulse = tat.correct_baseline(pulse, dpbl)

# 正負反転
pulse = -1.0 * pulse

# ------------------------------------
#  従来型の波形処理
# ------------------------------------

# 波形整形
pulse = tat.shaping(pulse, dt, 1.0e-6, 1.0e-6, True)

# 平均波形
avg = tat.make_average_pulse(pulse, 0.0, 1.0, 200, 500, True, True, True)

# 単純波高値
tat.simple_ph_spectrum(pulse, 200, 500, True, True)

# 最適フィルタ
ph, hist = tat.optimal_filter_freq(pulse, avg, noise, dt, 1.0e6, True, True)

# ------------------------------------
#  教師データ作成
# ------------------------------------

# 平均波形を規格化
avg = avg / np.max(avg)

# 最適フィルタの結果から、1光子当たりの波高値を得る
# 今回のデータではおよそ15mV
ph = 0.015

# 以下の方法で波形データをシミュレーションできる
# (規格化した平均波形) x (1光子当たりの波高値) x (光子数) + ノイズ
train_data = []
photon_num_list_train = []
noise_rate = 0.5
for _ in range(len(noise)):
    np.random.seed(_)
    num = int(7 * np.random.rand())
    simulation_data = avg * ph * num + noise[_, :] * noise_rate
    train_data.append(simulation_data)
    photon_num_list_train.append(num)
train_df = pd.DataFrame(train_data)
train_df["photon_count"] = photon_num_list_train

# テストデータ
test_df = pd.DataFrame(pulse)

# テストデータを曲線にフィッティングする場合
test_df = test_df.apply(regression, args=(time,), axis=1, result_type="expand")

# 教師ありクラスタリングモデル

In [24]:
class SupervisedClassificationModel:
    def __init__(self, train_df, test_df, features_create_method, classification_method):
        # train_df, test_df: 生パルスデータ
        self.train_df = train_df
        self.test_df = test_df
        self.train_y_df = train_df[["photon_count"]]
        self.features_create_method = features_create_method
        self.create_features(features_create_method)
        self.classification(classification_method)
        self.train_features_df = pd.concat([self.train_X_df, self.train_y_df], axis=1)
        self.test_features_df = pd.concat([self.test_X_df, self.test_y_df], axis=1)
        if self.features_create_method == "PCA":
            self.test_features_df = relabel_by_first_component_avg("y_pred", self.test_features_df)

    # 特徴量生成
    def create_features(self, features_create_method):
        # 主成分分析
        if features_create_method == "PCA":
            print("PCA processing ...")
            pca_model = PCA(n_components=2)
            train_X_pca = pca_model.fit_transform(self.train_df.iloc[:, :-1])
            test_X_pca = pca_model.transform(self.test_df)
            self.train_X_df = pd.DataFrame(train_X_pca, columns=["first_component", "second_component"])
            self.test_X_df = pd.DataFrame(test_X_pca, columns=["first_component", "second_component"])
        # tsfresh
        elif features_create_method == "tsfresh":
            print("tsfresh processing ...")
            tsfresh_features_train_file_path = "../data/tsfresh_features_train.csv"
            tsfresh_features_test_file_path = "../data/tsfresh_features_test.csv"
            if os.path.isfile(tsfresh_features_train_file_path) and os.path.isfile(tsfresh_features_test_file_path):
                extracted_features_train = pd.read_csv(tsfresh_features_train_file_path)
                extracted_features_test = pd.read_csv(tsfresh_features_test_file_path)
                print(f"shape of train features: {extracted_features_train.shape}") 
                print(f"shape of test features: {extracted_features_test.shape}") 
                common_features = extracted_features_train.columns.intersection(extracted_features_test.columns)
                print(f"selected features: {len(common_features)}")
                self.train_X_df = extracted_features_train[common_features]
                self.test_X_df = extracted_features_test[common_features]
            else:
                # tsfreshに対応した形に変換
                # train
                train_tsfresh = self.train_df.iloc[:, :-1].stack().reset_index()
                train_tsfresh.columns = ["id", "time", "voltage"]
                train_tsfresh["id"] = train_tsfresh["id"].astype("object")
                train_tsfresh["time"] = train_tsfresh["time"].astype("object")
                # test
                test_tsfresh = self.test_df.stack().reset_index()
                test_tsfresh.columns = ["id", "time", "voltage"]
                test_tsfresh["id"] = test_tsfresh["id"].astype("object")
                test_tsfresh["time"] = test_tsfresh["time"].astype("object")
                # tsfreshによる特徴量作成
                distributor_train = MultiprocessingDistributor(n_workers=os.cpu_count(), disable_progressbar=False, progressbar_title="Feature Extraction")
                extracted_features_train = extract_features(train_tsfresh, column_id="id", column_sort="time", distributor=distributor_train)
                distributor_test = MultiprocessingDistributor(n_workers=os.cpu_count(), disable_progressbar=False, progressbar_title="Feature Extraction")
                extracted_features_test = extract_features(test_tsfresh, column_id="id", column_sort="time", distributor=distributor_test)
                impute(extracted_features_train)
                impute(extracted_features_test)
                extracted_features_train = preprocess(extracted_features_train)
                extracted_features_test = preprocess(extracted_features_test)
                # csvファイルに保存
                extracted_features_train.to_csv("../data/tsfresh_features_train.csv", index=False)
                extracted_features_test.to_csv("../data/tsfresh_features_test.csv", index=False)
                print(f"shape of train features: {extracted_features_train.shape}") 
                print(f"shape of test features: {extracted_features_test.shape}")
                common_features = extracted_features_train.columns.intersection(extracted_features_test.columns)
                print(f"selected features: {len(common_features)}")
                self.train_X_df = extracted_features_train[common_features]
                self.test_X_df = extracted_features_test[common_features]

    # 分類
    def classification(self, classification_method):
        if classification_method == "logistic regression":
            model = LogisticRegression()
        elif classification_method == "decision tree":
            model = DecisionTreeClassifier()
        elif classification_method == "random forest":
            model = RandomForestClassifier()
        elif classification_method == "support vector machine":
            model = SVC()
        elif classification_method == "k neighbors classifier":
            model = KNeighborsClassifier()
        elif classification_method == "naive bayes":
            model = GaussianNB()
        print(f"training {classification_method} ...")
        model.fit(self.train_X_df.values, self.train_y_df.values)
        y_pred = model.predict(self.test_X_df.values)
        self.test_y_df = pd.DataFrame(y_pred, columns=["y_pred"])

    # testデータの光子数予測のヒストグラム
    def plot_hist(self):
        plt.hist(self.test_y_df["y_pred"], bins=128)
        plt.show()

    # 特徴量作成手法がPCAの場合の第1, 第2主成分のscatter
    def plot_pca_scatter(self):
        if self.features_create_method != "PCA":
            return
        for pred_label in set(self.test_y_df["y_pred"]):
            plt.scatter(
                self.test_features_df[self.test_features_df["y_pred"] == pred_label]["first_component"],
                self.test_features_df[self.test_features_df["y_pred"] == pred_label]["second_component"],
                s=2,
                label=pred_label,
            )
        plt.legend()
        plt.show()

In [15]:
# tsfreshでできる特徴量がtrainとtestで異なる
# train
train_tsfresh = train_df.iloc[:, :-1].stack().reset_index()
train_tsfresh.columns = ["id", "time", "voltage"]
train_tsfresh["id"] = train_tsfresh["id"].astype("object")
train_tsfresh["time"] = train_tsfresh["time"].astype("object")
display(train_tsfresh)
distributor_train = MultiprocessingDistributor(n_workers=os.cpu_count(), disable_progressbar=False, progressbar_title="Feature Extraction")
extracted_features_train = extract_features(train_tsfresh, column_id="id", column_sort="time", distributor=distributor_train)
print(f"shape 0: {extracted_features_train.shape}")
extracted_features_train_ = impute(extracted_features_train)
print(f"shape 1: {extracted_features_train_.shape}")
extracted_features_train__ = preprocess(extracted_features_train_)
print(f"shape 2: {extracted_features_train__.shape}")

,id,time,voltage
0,0,0,0.002079
1,0,1,0.001119
2,0,2,0.000799
3,0,3,0.001439
4,0,4,0.001119
...,...,...,...
10019995,9999,997,0.000470
10019996,9999,998,0.000155
10019997,9999,999,0.000801
10019998,9999,1000,0.001126


Feature Extraction: 100%|██████████| 60/60 [16:10<00:00, 16.18s/it]  


shape 0: (10000, 783)
shape 1: (10000, 783)
shape 2: (10000, 366)


In [17]:
# test
test_tsfresh = test_df.iloc[:, :-1].stack().reset_index()
test_tsfresh.columns = ["id", "time", "voltage"]
test_tsfresh["id"] = test_tsfresh["id"].astype("object")
test_tsfresh["time"] = test_tsfresh["time"].astype("object")
display(test_tsfresh)
distributor_test = MultiprocessingDistributor(n_workers=os.cpu_count(), disable_progressbar=False, progressbar_title="Feature Extraction")
extracted_features_test = extract_features(test_tsfresh, column_id="id", column_sort="time", distributor=distributor_test)
print(f"shape 0: {extracted_features_test.shape}")
extracted_features_test_ = impute(extracted_features_test)
print(f"shape 1: {extracted_features_test_.shape}")
extracted_features_test__ = preprocess(extracted_features_test_)
print(f"shape 2: {extracted_features_test__.shape}")

,id,time,voltage
0,0,0,0.000381
1,0,1,0.000213
2,0,2,0.000081
3,0,3,-0.000020
4,0,4,-0.000094
...,...,...,...
10009995,9999,996,0.000410
10009996,9999,997,0.000457
10009997,9999,998,0.000518
10009998,9999,999,0.000593


Feature Extraction: 100%|██████████| 60/60 [16:14<00:00, 16.25s/it]  


shape 0: (10000, 783)
shape 1: (10000, 783)
shape 2: (10000, 80)


In [25]:
my_model = SupervisedClassificationModel(train_df, test_df, features_create_method="tsfresh", classification_method="random forest")
my_model.plot_pca_scatter()

tsfresh processing ...
shape of train features: (10000, 366)
shape of test features: (10000, 80)
training random forest ...


,voltage__variation_coefficient,voltage__first_location_of_maximum,voltage__first_location_of_minimum,voltage__time_reversal_asymmetry_statistic__lag_3,voltage__autocorrelation__lag_0,voltage__partial_autocorrelation__lag_2,voltage__partial_autocorrelation__lag_3,voltage__partial_autocorrelation__lag_4,voltage__partial_autocorrelation__lag_5,voltage__partial_autocorrelation__lag_6,...,"voltage__linear_trend__attr_""pvalue""","voltage__agg_linear_trend__attr_""rvalue""__chunk_len_50__f_agg_""var""","voltage__agg_linear_trend__attr_""intercept""__chunk_len_50__f_agg_""min""","voltage__augmented_dickey_fuller__attr_""usedlag""__autolag_""AIC""",voltage__energy_ratio_by_chunks__num_segments_10__segment_focus_6,voltage__ratio_beyond_r_sigma__r_1.5,voltage__count_below__t_0,voltage__fourier_entropy__bins_100,voltage__mean_n_absolute_max__number_of_maxima_7,y_pred
0,-0.020513,-0.123250,-0.072144,-0.106119,0.000000e+00,-0.003261,0.007136,0.382580,-0.001997,-0.009927,...,-0.232775,-0.169683,-1.702004,-1.640705,-0.569632,0.252642,1.143738,-0.460296,0.373489,4
1,-0.019383,-0.416540,-2.978244,-0.180948,0.000000e+00,-0.203955,-0.055845,-0.007363,0.002279,-0.009660,...,-0.314765,-0.714348,-0.605534,0.569594,-0.825191,-0.612938,-0.570677,2.059917,-1.237785,0
2,0.032238,3.887319,2.148993,0.264475,0.000000e+00,-0.056156,-0.036884,-0.000622,0.004146,-0.009125,...,-0.314765,1.661249,0.657905,0.569594,1.509348,-0.448066,0.147865,2.059917,-1.173215,0
3,-0.031354,-0.157353,-0.005643,-0.090316,0.000000e+00,-0.018241,-0.017438,0.010289,0.008642,-0.006242,...,-0.314756,-0.170238,-0.031048,0.569594,0.182942,0.829696,-0.381587,-0.460296,0.766420,5
4,-0.019482,-0.027760,-1.701422,-0.054251,0.000000e+00,-0.067368,-0.044621,-0.005774,0.002212,-0.009801,...,-0.208426,1.272821,-0.222419,0.569594,0.586247,0.417515,-0.444617,-0.460296,-1.156265,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,-0.016648,-0.130071,0.047558,-0.159646,0.000000e+00,-0.013095,-0.011051,0.019387,0.034687,-0.011260,...,-0.314765,0.670414,1.472534,0.569594,-0.918166,-0.324411,-1.516127,-0.460296,-0.760272,2
9996,-0.016323,-0.177816,-0.012293,-0.146929,2.220446e-16,-0.011608,-0.009087,0.023336,-0.375433,-0.010755,...,-0.314765,1.825723,1.712167,0.569594,-0.129947,0.499951,-0.999281,-0.460296,-0.786279,2
9997,-0.020550,-0.136891,-0.072144,0.401022,0.000000e+00,-0.005914,0.001458,0.081799,-0.003866,-0.010031,...,-0.313925,-0.454877,-1.815524,0.569594,-0.232102,-0.118321,0.664710,-0.460296,0.677293,4
9998,-0.020576,-0.136891,0.533016,-0.220414,0.000000e+00,-0.011199,-0.008693,0.024110,-0.117379,-0.010697,...,-0.314644,-0.264289,-0.692558,0.569594,1.451275,-2.385318,0.954648,-0.460296,0.209416,3
